In [1]:
import torch
x = torch.randn(5, 5)
x

tensor([[ 0.3603, -1.5605,  0.9497, -0.2348, -0.0369],
        [ 0.3642,  0.7602, -1.9142,  0.1980,  0.4829],
        [-0.8092, -0.2852,  0.5729, -0.3688,  0.8005],
        [-1.6383,  0.4493, -0.4225,  0.7692,  2.2470],
        [-0.7717,  0.1226,  0.0510, -0.6089,  0.4468]])

In [3]:
y = x.unsqueeze(0)
y, y.shape

(tensor([[[ 0.3603, -1.5605,  0.9497, -0.2348, -0.0369],
          [ 0.3642,  0.7602, -1.9142,  0.1980,  0.4829],
          [-0.8092, -0.2852,  0.5729, -0.3688,  0.8005],
          [-1.6383,  0.4493, -0.4225,  0.7692,  2.2470],
          [-0.7717,  0.1226,  0.0510, -0.6089,  0.4468]]]),
 torch.Size([1, 5, 5]))

In [9]:
distance = torch.tensor([1,2,3,4,5,6,7,8])
torch.exp(-distance / 1), torch.exp(-distance / 2)

(tensor([3.6788e-01, 1.3534e-01, 4.9787e-02, 1.8316e-02, 6.7379e-03, 2.4788e-03,
         9.1188e-04, 3.3546e-04]),
 tensor([0.6065, 0.3679, 0.2231, 0.1353, 0.0821, 0.0498, 0.0302, 0.0183]))

In [ ]:
import argparse
import torch
import torch.nn.functional as F
import numpy as np
import yaml
import os
import os.path as osp
from yaml import SafeLoader
from Data.get_data import get_data
from run import run
from run_batch import run_batch
from Data.data_utils import load_fixed_splits
import random
import warnings


def fix_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True


if __name__ == '__main__':
    warnings.filterwarnings('ignore')
    parser = argparse.ArgumentParser()
    parser.add_argument('--dataset', type=str, default='Cora')
    parser.add_argument('--learning_rate', type=float, default=0.01)
    parser.add_argument('--weight_decay', type=float, default=5e-4)
    parser.add_argument('--gcn_wd', type=float, default=5e-4)
    parser.add_argument('--gpu_id', type=int, default=6)
    parser.add_argument('--config', type=str, default='config.yaml')
    parser.add_argument('--gcn_use_bn', action='store_true', help='gcn use batch norm')
    parser.add_argument('--show_details', type=bool, default=True)
    parser.add_argument('--gcn_type', type=int, default=1)
    parser.add_argument('--gcn_layers', type=int, default=2)
    parser.add_argument('--batch_size', type=int, default=100000)
    parser.add_argument('--rand_split', action='store_true', help='random split dataset')
    parser.add_argument('--rand_split_class', action='store_true', help='random split dataset by class')
    parser.add_argument('--protocol', type=str, default='semi')
    parser.add_argument('--label_num_per_class', type=int, default=20)
    parser.add_argument('--train_prop', type=float, default=.6)
    parser.add_argument('--valid_prop', type=float, default=.2)
    parser.add_argument('--alpha', type=float, default=.8)
    parser.add_argument('--tau', type=float, default=.3)

    args = parser.parse_args()

    assert args.gpu_id in range(0, 8)
    device = torch.device('mps' if torch.backends.mps.is_available() else 'cpu')

    config = yaml.load(open(args.config), Loader=SafeLoader)[args.dataset]
    fix_seed(config['seed'])

    # path = osp.join(osp.expanduser('~'), 'datasets/')
    path = '/Users/jihaoran/PycharmProjects/CoBFormer/Data'

    results = dict()
    alpha = args.alpha
    tau = args.tau

    # postfix = f'{n_patch}'
    postfix = "test"
    runs = 5

    data = get_data(path, args.dataset)
    
    # calculate calibration mask
    # step 1: calculate shortest path distance matrix
    # step 2: calculate calibration mask: exp(-distance/tau)
    data.graph['calibration_mask'] = None
    
    # get the splits for all runs
    if args.rand_split:
        split_idx_lst = [data.get_idx_split(train_prop=args.train_prop, valid_prop=args.valid_prop)
                         for _ in range(runs)]
    elif args.rand_split_class:
        split_idx_lst = [data.get_idx_split(split_type='class', label_num_per_class=args.label_num_per_class)
                         for _ in range(runs)]
    else:
        split_idx_lst = load_fixed_splits(path, data, name=args.dataset, protocol=args.protocol)

    batch_size = args.batch_size

    results = [[], []]
    for r in range(runs):
        if args.dataset in ['Cora', 'CiteSeer', 'PubMed', 'ogbn-arxiv', 'ogbn-products'] and args.protocol == 'semi':
            split_idx = split_idx_lst[0]
        else:
            split_idx = split_idx_lst[r]

        if args.dataset in ['ogbn-products']:
            res_gnn, res_trans = run_batch(args, config, device, data, batch_size, split_idx, alpha, tau,
                                           postfix)
        else:
            res_gnn, res_trans = run(args, config, device, data, split_idx, alpha, tau, postfix)
        results[0].append(res_gnn)
        results[1].append(res_trans)

    print(f"==== Final GNN====")
    result = torch.tensor(results[0]) * 100.
    print(result)
    print(f"max: {torch.max(result, dim=0)[0]}")
    print(f"min: {torch.min(result, dim=0)[0]}")
    print(f"mean: {torch.mean(result, dim=0)}")
    print(f"std: {torch.std(result, dim=0)}")

    print(f'GNN Micro: {torch.mean(result, dim=0)[1]:.2f} ± {torch.std(result, dim=0)[1]:.2f}')
    print(f'GNN Macro: {torch.mean(result, dim=0)[3]:.2f} ± {torch.std(result, dim=0)[3]:.2f}')

    print(f"==== Final Trans====")
    result = torch.tensor(results[1]) * 100.
    print(result)
    print(f"max: {torch.max(result, dim=0)[0]}")
    print(f"min: {torch.min(result, dim=0)[0]}")
    print(f"mean: {torch.mean(result, dim=0)}")
    print(f"std: {torch.std(result, dim=0)}")

    print(f'Trans Micro: {torch.mean(result, dim=0)[1]:.2f} ± {torch.std(result, dim=0)[1]:.2f}')
    print(f'Trans Macro: {torch.mean(result, dim=0)[3]:.2f} ± {torch.std(result, dim=0)[3]:.2f}')


In [1]:
import numpy as np

# 创建两个形状相同的矩阵
matrix1 = np.array([[1, 2], [3, 4]])
matrix2 = np.array([[5, 6], [7, 8]])

# 对应位置的元素相乘
result = matrix1 * matrix2

print("矩阵1:")
print(matrix1)
print("矩阵2:")
print(matrix2)
print("对应位置元素相乘的结果:")
print(result)

矩阵1:
[[1 2]
 [3 4]]
矩阵2:
[[5 6]
 [7 8]]
对应位置元素相乘的结果:
[[ 5 12]
 [21 32]]


In [1]:
import networkx as nx
import numpy as np

def floyd_warshall_shortest_path(G, num_nodes):
    # 初始化距离矩阵
    shortest_path_matrix = np.full((num_nodes, num_nodes), np.inf)
    for i in range(num_nodes):
        shortest_path_matrix[i, i] = 0
    for u, v in G.edges():
        shortest_path_matrix[u, v] = 1
        shortest_path_matrix[v, u] = 1

    # Floyd-Warshall算法核心
    for k in range(num_nodes):
        for i in range(num_nodes):
            for j in range(num_nodes):
                shortest_path_matrix[i, j] = min(
                    shortest_path_matrix[i, j],
                    shortest_path_matrix[i, k] + shortest_path_matrix[k, j]
                )

    return shortest_path_matrix

# 示例使用
if __name__ == "__main__":
    # 创建一个示例图
    G = nx.Graph()
    G.add_edges_from([(0, 1), (1, 2), (2, 3)])
    num_nodes = G.number_of_nodes()

    # 计算最短路径矩阵
    shortest_path_matrix = floyd_warshall_shortest_path(G, num_nodes)
    print("最短路径矩阵:")
    print(shortest_path_matrix)  

最短路径矩阵:
[[0. 1. 2. 3.]
 [1. 0. 1. 2.]
 [2. 1. 0. 1.]
 [3. 2. 1. 0.]]
